In [3]:
import json
import numpy as np
import onnxruntime as ort
from PIL import Image

In [10]:
def preprocess(input_data):
    # convert the input data into the float32 input
    img_data = input_data.astype('float32')

    #normalize
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]

    #add batch channel
    norm_img_data = norm_img_data.reshape(1, 3, 224, 224).astype('float32') 
    return norm_img_data

In [16]:
image_path = input("Enter image path: ")
image = Image.open(image_path).resize((224, 224))
image_data = np.array(image).transpose(2, 0, 1).astype('float32')
image_data = np.expand_dims(image_data, axis=0)
image_data = preprocess(image_data)
print(image_data.shape)

(1, 3, 224, 224)


In [12]:
onnx_path = 'resnet50v2/resnet50v2.onnx'
session = ort.InferenceSession(onnx_path)

In [17]:
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
result = session.run([output_name], {input_name: image_data})[0]

In [14]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [18]:
p=softmax(result[0]).tolist()
with open('imagenet-simple-labels.json') as f:
    labels = json.load(f)
    arr = np.array(p).argsort()[-5:][::-1]
    for id in arr:
        print(labels[id], p[id])

convertible 0.3483785390853882
pickup truck 0.26876357197761536
sports car 0.12649530172348022
station wagon 0.07169000059366226
car wheel 0.05167302116751671
